## Overview

We wanted to determine if analysis of Worldwide Covid-19 vaccination progress alongside Worldwide case data would allow us to observe a noticable relationship between the two. We examined relationships both on a global scale and on a country-by-country basis. A linear regression test was used to examine our results.

In this notebook is;
- Dataset import from csv
- Dataset conversion to dataframe
- Dataframe cleaning and building for analysis

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [3]:
# Creating df from both the csv files
covid_cases_df = pd.read_csv('Resource/covid-data.csv')
covid_vacc_df = pd.read_csv('Resource/country_vaccinations.csv')


# updating the df to have columns needed for analysis
covid_vacc_df = covid_vacc_df[['country', 'date', 'iso_code', 'people_vaccinated', \
                               'daily_vaccinations', 'people_vaccinated_per_hundred']]

# converting date object to pandas 'datetime64[ns]' format
covid_vacc_df['date'] = pd.to_datetime(covid_vacc_df['date'])

covid_cases_df = covid_cases_df[['location', 'date', 'iso_code', 'total_cases', 'new_cases', \
                                'total_deaths', 'new_deaths', 'population', 'total_cases_per_million', \
                                 'new_cases_per_million']]

# converting date object to pandas 'datetime64[ns]' format
covid_cases_df['date'] = pd.to_datetime(covid_cases_df['date'])

In [4]:
# Adding total cases per hundred column
covid_cases_df['total_cases_per_hundred'] = (covid_cases_df['total_cases'] / covid_cases_df['population']) * 100

covid_cases_df = covid_cases_df.reset_index(drop=True)

In [5]:
# Grouped max vaccination DF on country
max_covid_vacc_df = covid_vacc_df.groupby(['iso_code'], as_index=False).max().\
                        sort_values(by=['people_vaccinated_per_hundred'], ascending=False).reset_index(drop=True)
max_covid_vacc_df.head()

,iso_code,country,date,people_vaccinated,daily_vaccinations,people_vaccinated_per_hundred
0,GIB,Gibraltar,2022-03-24,42034.0,1068.0,124.76
1,NIU,Niue,2022-03-13,1650.0,87.0,102.23
2,PCN,Pitcairn,2021-09-07,47.0,1.0,100.00
3,ARE,United Arab Emirates,2022-03-29,9890318.0,155312.0,98.99
4,PRT,Portugal,2022-03-10,9663542.0,150867.0,95.04


In [6]:
covid_cases_df.head()

,location,date,iso_code,total_cases,new_cases,total_deaths,new_deaths,population,total_cases_per_million,new_cases_per_million,total_cases_per_hundred
0,Afghanistan,2020-02-24,AFG,5.0,5.0,NaN,NaN,39835428.0,0.126,0.126,0.000013
1,Afghanistan,2020-02-25,AFG,5.0,0.0,NaN,NaN,39835428.0,0.126,0.000,0.000013
2,Afghanistan,2020-02-26,AFG,5.0,0.0,NaN,NaN,39835428.0,0.126,0.000,0.000013
3,Afghanistan,2020-02-27,AFG,5.0,0.0,NaN,NaN,39835428.0,0.126,0.000,0.000013
4,Afghanistan,2020-02-28,AFG,5.0,0.0,NaN,NaN,39835428.0,0.126,0.000,0.000013


In [7]:
# Grouped max covid cases DF on country
# Sorting the DF based on total cases per hundred
# Dropping the rows with NAN values for total cases and resetting the index

max_covid_case_df = covid_cases_df.groupby(['iso_code'], as_index=False).max()

sort_covid_case_per_hund_df = max_covid_case_df.drop(['new_cases', 'new_deaths', 'new_cases_per_million'], \
                           axis=1).sort_values(by=['total_cases_per_hundred'], ascending=False).dropna(subset=['total_cases']).reset_index(drop=True)


# Sorting the DF based on total cases
sort_most_covid_case_df = max_covid_case_df.drop(['new_cases', 'new_deaths', 'new_cases_per_million'], \
                           axis=1).sort_values(by=['total_cases'], ascending=False).dropna(subset=['total_cases']).reset_index(drop=True)

In [8]:
# This df shows the most effected countries based on cases per hundred. 
sort_covid_case_per_hund_df.head()

,iso_code,location,date,total_cases,total_deaths,population,total_cases_per_million,total_cases_per_hundred
0,FRO,Faeroe Islands,2022-03-05,34658.0,28.0,49053.0,706541.904,70.654190
1,AND,Andorra,2022-03-05,38434.0,151.0,77354.0,496858.598,49.685860
2,DNK,Denmark,2022-03-05,2853236.0,4830.0,5813302.0,490811.590,49.081159
3,GIB,Gibraltar,2022-03-05,15622.0,101.0,33691.0,463684.664,46.368466
4,SVN,Slovenia,2022-03-05,902744.0,6351.0,2078723.0,434278.160,43.427816


In [9]:
# This df shows the most effected countries based on most number of total cases. World and Continental data are excluded.
sort_most_covid_case_df = sort_most_covid_case_df[~sort_most_covid_case_df['iso_code'].str.contains('OWID')]
sort_most_covid_case_df

,iso_code,location,date,total_cases,total_deaths,population,total_cases_per_million,total_cases_per_hundred
8,USA,United States,2022-03-05,79265726.0,958437.0,3.329151e+08,238095.936,23.809594
10,IND,India,2022-03-05,42962953.0,515036.0,1.393409e+09,30832.980,3.083298
11,BRA,Brazil,2022-03-05,29040800.0,652216.0,2.139934e+08,135708.832,13.570883
12,FRA,France,2022-03-05,23064766.0,139312.0,6.742200e+07,342095.547,34.209555
13,GBR,United Kingdom,2022-03-05,19172095.0,162152.0,6.820711e+07,281086.442,28.108644
...,...,...,...,...,...,...,...,...
223,VUT,Vanuatu,2022-03-05,19.0,1.0,3.144640e+05,60.420,0.006042
224,MHL,Marshall Islands,2022-03-05,7.0,NaN,5.961800e+04,117.414,0.011741
225,SHN,Saint Helena,2022-03-05,4.0,NaN,6.095000e+03,656.276,0.065628
226,COK,Cook Islands,2022-03-05,2.0,NaN,1.757200e+04,113.817,0.011382


In [10]:
#merging two dfs on 'iso-code' and 'date'

merge_df = pd.merge(covid_cases_df, covid_vacc_df, on=['iso_code', 'date'], how='outer')

In [11]:
#verfying the merge df

merge_df.loc[(merge_df['iso_code'] == 'AFG') & (merge_df['date'] == '2022-03-22')]

,location,date,iso_code,total_cases,new_cases,total_deaths,new_deaths,population,total_cases_per_million,new_cases_per_million,total_cases_per_hundred,country,people_vaccinated,daily_vaccinations,people_vaccinated_per_hundred
166342,NaN,2022-03-22,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,5082824.0,6319.0,12.76


In [14]:
#creating dataframe to show top 30 largest countries
large_countries_df = merge_df.loc[(merge_df['population'] > 47000000) & (~merge_df['iso_code'].str.contains('OWID')), :]

large_countries_df['location'].nunique()

30

In [17]:
sorted_large_df = large_countries_df.groupby(['iso_code'], as_index=False).max(). \
                    sort_values(by=['people_vaccinated_per_hundred'], ascending=False).reset_index(drop = True)
sorted_large_df

D:\Users\Jacob M\AppData\Local\Temp\ipykernel_512\3610958560.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  sorted_large_df = large_countries_df.groupby(['iso_code'], as_index=False).max(). \


,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,population,total_cases_per_million,new_cases_per_million,total_cases_per_hundred,people_vaccinated,daily_vaccinations,people_vaccinated_per_hundred
0,CHN,China,2022-03-05,110868.0,15133.0,4654.0,1290.0,1.444216e+09,76.767,10.478,0.007677,1.269302e+09,22424286.0,87.89
1,KOR,South Korea,2022-03-05,4456264.0,266838.0,8957.0,216.0,5.130518e+07,86857.967,5200.995,8.685797,4.488254e+07,880695.0,87.48
2,ITA,Italy,2022-03-05,12990223.0,228123.0,155782.0,993.0,6.036747e+07,215185.808,3778.906,21.518581,5.065603e+07,665762.0,83.91
3,BRA,Brazil,2022-03-05,29040800.0,287149.0,652216.0,4148.0,2.139934e+08,135708.832,1341.859,13.570883,1.782039e+08,1941268.0,83.28
4,VNM,Vietnam,2022-03-05,4232520.0,176150.0,40726.0,804.0,9.816883e+07,43114.704,1794.358,4.311470,7.959860e+07,1675471.0,81.08
5,COL,Colombia,2022-03-05,6070616.0,35575.0,139037.0,720.0,5.126584e+07,118414.443,693.932,11.841444,4.140938e+07,414465.0,80.77
6,JPN,Japan,2022-03-05,5341246.0,104345.0,24816.0,322.0,1.260508e+08,42373.759,827.801,4.237376,1.017601e+08,1997542.0,80.73
7,FRA,France,2022-03-05,23064766.0,502507.0,139312.0,1438.0,6.742200e+07,342095.547,7453.161,34.209555,5.396583e+07,733656.0,80.04
8,GBR,United Kingdom,2022-03-05,19172095.0,219290.0,162152.0,1820.0,6.820711e+07,281086.442,3215.061,28.108644,5.266891e+07,970350.0,77.22
9,THA,Thailand,2022-03-05,3026695.0,42617.0,23235.0,312.0,6.995084e+07,43268.885,609.242,4.326888,5.393852e+07,846179.0,77.11


In [13]:
world_cases_df  = covid_cases_df.loc[covid_cases_df['iso_code'] == 'OWID_WRL']
world_cases_df.head()

,location,date,iso_code,total_cases,new_cases,total_deaths,new_deaths,population,total_cases_per_million,new_cases_per_million,total_cases_per_hundred
163423,World,2020-01-22,OWID_WRL,557.0,0.0,17.0,0.0,7.874966e+09,0.071,0.000,0.000007
163424,World,2020-01-23,OWID_WRL,657.0,100.0,18.0,1.0,7.874966e+09,0.083,0.013,0.000008
163425,World,2020-01-24,OWID_WRL,944.0,287.0,26.0,8.0,7.874966e+09,0.120,0.036,0.000012
163426,World,2020-01-25,OWID_WRL,1437.0,493.0,42.0,16.0,7.874966e+09,0.182,0.063,0.000018
163427,World,2020-01-26,OWID_WRL,2120.0,683.0,56.0,14.0,7.874966e+09,0.269,0.087,0.000027


In [17]:
world_covid_vacc = covid_vacc_df.groupby(['date'], as_index=False).sum()
world_covid_vacc

,date,people_vaccinated,daily_vaccinations,people_vaccinated_per_hundred
0,2020-12-02,0.000000e+00,0.0,0.00
1,2020-12-03,0.000000e+00,0.0,0.00
2,2020-12-04,1.000000e+00,0.0,0.00
3,2020-12-05,0.000000e+00,0.0,0.00
4,2020-12-06,0.000000e+00,0.0,0.00
...,...,...,...,...
478,2022-03-25,2.764019e+09,13179271.0,6253.19
479,2022-03-26,2.781163e+09,13672723.0,4730.37
480,2022-03-27,2.945374e+09,13764003.0,4574.25
481,2022-03-28,2.696730e+09,14386042.0,4432.75


In [18]:
world_df = pd.merge(world_cases_df, world_covid_vacc, on='date', how='inner')
world_df

,location,date,iso_code,total_cases,new_cases,total_deaths,new_deaths,population,total_cases_per_million,new_cases_per_million,total_cases_per_hundred,people_vaccinated,daily_vaccinations,people_vaccinated_per_hundred
0,World,2020-12-02,OWID_WRL,64668183.0,646764.0,1551318.0,12643.0,7.874966e+09,8211.868,82.129,0.821187,0.000000e+00,0.0,0.00
1,World,2020-12-03,OWID_WRL,65360343.0,692160.0,1563918.0,12600.0,7.874966e+09,8299.762,87.894,0.829976,0.000000e+00,0.0,0.00
2,World,2020-12-04,OWID_WRL,66049040.0,688697.0,1576378.0,12460.0,7.874966e+09,8387.216,87.454,0.838722,1.000000e+00,0.0,0.00
3,World,2020-12-05,OWID_WRL,66697974.0,648934.0,1586715.0,10337.0,7.874966e+09,8469.621,82.405,0.846962,0.000000e+00,0.0,0.00
4,World,2020-12-06,OWID_WRL,67234273.0,536299.0,1594444.0,7729.0,7.874966e+09,8537.723,68.102,0.853772,0.000000e+00,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,World,2022-03-01,OWID_WRL,438528632.0,1536867.0,5964706.0,8069.0,7.874966e+09,55686.418,195.159,5.568642,2.919238e+09,21914705.0,5707.48
455,World,2022-03-02,OWID_WRL,440180084.0,1651452.0,5972651.0,7945.0,7.874966e+09,55896.127,209.709,5.589613,2.831305e+09,21250899.0,5875.52
456,World,2022-03-03,OWID_WRL,442067549.0,1887465.0,5981034.0,8384.0,7.874966e+09,56135.806,239.679,5.613581,2.714213e+09,20217862.0,5095.32
457,World,2022-03-04,OWID_WRL,443767689.0,1704138.0,5989802.0,8771.0,7.874966e+09,56351.698,216.399,5.635170,2.709618e+09,19276636.0,5517.04


In [16]:
# Dataframes available to use

#1. covid_vacc_df  ---- raw df with required columns for vaccination record
#2. covid_cases_df ---- raw df with required columns for cases record with added column for "total cases per hundred"
                        # plus data for world and continents are removed from it
    
#3. max_covid_vacc_df ---- data is grouped on country and is sorted with the countries with most people vaccinated
#4. sort_covid_case_per_hund_df ---- data is grouped on country and is sorted with the countries with most cases per hundred
#5. sort_most_covid_case_df ---- data is grouped on country and is sorted with the countries with most cases
#6. merge_df ---- its a merged df 'on=['iso_code', 'date']' containg vaccination and cases record. 
#7. large_countries_df ----  dataframe to show top 30 largest countries based on population
#8. world_df ---- dataframe contains daily cases and daily vaccinations for the world
